#*1) clone repo*




In [1]:
!git clone https://github.com/womeramichay/elta_home_assignment

%cd elta_home_assignment


Cloning into 'elta_home_assignment'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 58 (delta 24), reused 6 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (58/58), 79.84 KiB | 11.41 MiB/s, done.
Resolving deltas: 100% (24/24), done.
/content/elta_home_assignment


---
# **2)Kaggle auth**

In [2]:

!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json


cp: cannot stat '/content/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


---
# **3) Download the data**

In [3]:
!mkdir -p data/raw
!kaggle competitions download -c titanic -p data/raw
!unzip -o data/raw/titanic.zip -d data/raw


Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 4, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.12/dist-packages/kaggle/__init__.py", line 6, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 434, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/
unzip:  cannot find or open data/raw/titanic.zip, data/raw/titanic.zip.zip or data/raw/titanic.zip.ZIP.


---

#**4) quick look at the data.**




In [4]:
from pathlib import Path
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 200)

# data downloaded via Kaggle into data/raw/
DATA_PATH = Path("data/raw/train.csv")

df = pd.read_csv(DATA_PATH)

df.head(10)


FileNotFoundError: [Errno 2] No such file or directory: 'data/raw/train.csv'

In [ ]:
# test size
df["Survived"].value_counts()


In [ ]:
# test distribution
df["Survived"].value_counts(normalize=True)

# #missing values by col


In [ ]:
(df.isna().mean() * 100).sort_values(ascending=False)

In [ ]:
df.dtypes


# **Cross corelation for numeric cols**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Select numeric columns including target
numeric_cols = [
    "Survived",
    "Pclass",
    "Age",
    "SibSp",
    "Parch",
    "Fare",
]

corr = df[numeric_cols].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation matrix (numeric features)")
plt.show()


#Feature selection decisions
---
1) After reviewing the EDA results and the correlation heatmap, I decided to drop the following columns:


*   PassengerId - Identifier only an arbitrary value that have no corelation to survival rate.
*   Name - a text that have no direct correlation to survival rate.
*   Ticket - a text that have no clear stracture, not realy learnable.
*   Cabin - missing to many values.

2) Handling missing values::
  * age : Missing values are imputed using the median (obust to skewed distributions).
  * Embarked: Missing values are filled with the most frequent category before encoding, to avoid introducing an implicit “missing” category.


3) Categorical encoding: Non-numeric categorical features (**Sex**, **Embarked**) are represented using one-hot encoding.







# Select features to keep and creat sample + lables + One hot encode categorical features

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# fix categorical missing values (explicit, clean)
df = df.copy()
df["Embarked"] = df["Embarked"].fillna(df["Embarked"].mode()[0])

# Select the features we keep (dropping others by omission)
features = ["Pclass", "Age", "SibSp", "Parch", "Fare", "Sex", "Embarked"]
X = df[features]
y = df["Survived"]

# one hot encode categorical columns
X_encoded = pd.get_dummies(X,columns=["Sex", "Embarked"],drop_first=True)

# impute numeric missing values (Age, Fare if needed)
numeric_cols = ["Pclass", "Age", "SibSp", "Parch", "Fare"]
num_imputer = SimpleImputer(strategy="median")
X_encoded[numeric_cols] = num_imputer.fit_transform(X_encoded[numeric_cols])

# train-validation split (stratified)
X_train, X_val, y_train, y_val = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42, stratify=y)

# sanity checks
print("X_train shape:", X_train.shape)
print("Any NaNs left?", X_train.isna().any().any(), X_val.isna().any().any())
# print("Encoded columns:", list(X_encoded.columns))

X_encoded.head()


---
#**Simple train using Logistic Regretion BASLINE**
just to have a baseline and see F1 score.
it will help understand which  new features to generate.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

def eval_logreg_f1(X, y):
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)
    return f1_score(y_val, y_pred)


In [ ]:
baseline_f1 = eval_logreg_f1(X_encoded, y)
print("baseline_f1:", round(baseline_f1, 4))




#looking at SVM BASLINE

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

def eval_svm_f1(X, y):
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    model = LinearSVC(max_iter=5000)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)
    return f1_score(y_val, y_pred)


In [ ]:
logreg_f1 = eval_logreg_f1(X_encoded, y)
svm_f1 = eval_svm_f1(X_encoded, y)

print("Logistic Regression F1:", round(logreg_f1, 4))
print("Linear SVM F1:        ", round(svm_f1, 4))
print("Delta (SVM - LR):     ", round(svm_f1 - logreg_f1, 4))


# Here i will test some hypotheses about new features selection

In [ ]:
# Target statistics: survival rate by group

def survival_rate(col):
    return df.groupby(col)["Survived"].mean().sort_values(ascending=False)

survival_rate("Sex")
survival_rate("Pclass")
survival_rate("Embarked")


In [ ]:
# Age bins → survival rate
df["AgeBin"] = pd.cut(
    df["Age"],
    bins=[0, 12, 18, 35, 60, 100],
    labels=["Child", "Teen", "YoungAdult", "Adult", "Senior"]
)

survival_rate("AgeBin")


In [ ]:
def survival_rate(col):
    return df.groupby(col)["Survived"].mean().sort_values(ascending=False)

# Inspect categorical signals
print("Sex:\n", survival_rate("Sex"), "\n")
print("Pclass:\n", survival_rate("Pclass"), "\n")
print("Embarked:\n", survival_rate("Embarked"), "\n")


In [ ]:
df["IsChild"] = (df["Age"] < 18).astype(int)

survival_rate("IsChild")


In [ ]:
X_child = X_encoded.copy()
X_child["IsChild"] = df["IsChild"].fillna(0)

child_f1 = eval_logreg_f1(X_child, y)

print(f"baseline_f1: {baseline_f1:.4f}")
print(f"child_f1:    {child_f1:.4f}")
print(f"delta:       {child_f1 - baseline_f1:.4f}")


In [ ]:
#  Sex × Pclass interaction (new cell)
X_inter = X_encoded.copy()

# create 3 interaction columns: male AND each Pclass
# (works because you already have: Pclass numeric + Sex_male dummy)
for p in [1, 2, 3]:
    X_inter[f"SexMale_x_Pclass{p}"] = (X_inter["Sex_male"].astype(int) * (X_inter["Pclass"] == p).astype(int))

# 3) evaluate using the SAME function + baseline_f1 you already computed
inter_f1 = eval_logreg_f1(X_inter, y)

print(f"baseline_f1: {baseline_f1:.4f}")
print(f"inter_f1:    {inter_f1:.4f}")
print(f"delta:       {inter_f1 - baseline_f1:.4f}")

# quick sanity (optional): see the new columns exist
X_inter[[c for c in X_inter.columns if c.startswith("SexMale_x_")]].head()


In [ ]:
# NEW FEATURE TEST: IsAlone

X_alone = X_encoded.copy()

# Create IsAlone from SibSp and Parch
X_alone["IsAlone"] = ((X_alone["SibSp"] + X_alone["Parch"]) == 0).astype(int)

# Evaluate
alone_f1 = eval_logreg_f1(X_alone, y)

print(f"baseline_f1: {baseline_f1:.4f}")
print(f"isalone_f1:  {alone_f1:.4f}")
print(f"delta:       {alone_f1 - baseline_f1:.4f}")


#Random forest basrline

In [ ]:
# --- MODEL CHANGE TEST: Random Forest (new cell) ---

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# same split logic as before
X_train, X_val, y_train, y_val = train_test_split(
    X_encoded,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)
y_pred = rf.predict(X_val)

rf_f1 = f1_score(y_val, y_pred)

print(f"logreg_baseline_f1: {baseline_f1:.4f}")
print(f"random_forest_f1:   {rf_f1:.4f}")
print(f"delta:              {rf_f1 - baseline_f1:.4f}")


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# same split for fair comparison
X_train, X_val, y_train, y_val = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42, stratify=y
)

rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=6,          # <-- key change (was too shallow before)
    min_samples_leaf=1,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)
y_pred = rf.predict(X_val)

rf_f1 = f1_score(y_val, y_pred)

print(f"logreg_baseline_f1: {baseline_f1:.4f}")
print(f"random_forest_f1:  {rf_f1:.4f}")
print(f"delta:             {rf_f1 - baseline_f1:.4f}")


### Model selection conclusion

Several engineered features (IsChild, IsAlone, interaction terms) were evaluated
using controlled F1 comparisons and did not improve performance.

A Random Forest model was tested to capture potential non-linear interactions,
but it underperformed the Logistic Regression baseline, indicating that the
remaining predictive signal is largely linear and already well captured.

Therefore, Logistic Regression will be our baseline model.


#NN Model

In [ ]:
import numpy as np
import tensorflow as tf

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score


def _make_mlp(input_dim,
              hidden_units=(128, 64),
              activation="relu",          # "relu" | "gelu" | "leakyrelu"
              dropout=0.3,
              batch_norm=True,
              optimizer_name="adam",      # "adam" | "adamw" | "sgd"
              lr=1e-3,
              weight_decay=1e-4):
    inputs = tf.keras.Input(shape=(input_dim,))
    x = inputs

    for units in hidden_units:
        x = tf.keras.layers.Dense(units)(x)
        if batch_norm:
            x = tf.keras.layers.BatchNormalization()(x)

        if activation == "leakyrelu":
            x = tf.keras.layers.LeakyReLU(alpha=0.1)(x)
        else:
            x = tf.keras.layers.Activation(activation)(x)  # relu / gelu

        if dropout and dropout > 0:
            x = tf.keras.layers.Dropout(dropout)(x)

    outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    model = tf.keras.Model(inputs, outputs)

    opt = optimizer_name.lower()
    if opt == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif opt == "adamw":
        optimizer = tf.keras.optimizers.AdamW(learning_rate=lr, weight_decay=weight_decay)
    elif opt == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9)
    else:
        raise ValueError("optimizer_name must be one of: adam, adamw, sgd")

    model.compile(optimizer=optimizer, loss="binary_crossentropy")
    return model


def eval_nn_cv_f1(X, y,
                  optimizer_name="adam",
                  lr=1e-3,
                  hidden_units=(128, 64),
                  activation="relu",
                  dropout=0.3,
                  batch_norm=True,
                  weight_decay=1e-4,
                  epochs=80,
                  batch_size=64,
                  patience=10,
                  n_splits=5,
                  random_state=42,
                  verbose=0):
    X = np.asarray(X, dtype=np.float32)
    y = np.asarray(y, dtype=np.int32)

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    scores = []

    for tr_idx, va_idx in skf.split(X, y):
        X_tr, X_va = X[tr_idx], X[va_idx]
        y_tr, y_va = y[tr_idx], y[va_idx]

        # scale (fit only on train fold)
        scaler = StandardScaler()
        X_tr = scaler.fit_transform(X_tr).astype(np.float32)
        X_va = scaler.transform(X_va).astype(np.float32)

        model = _make_mlp(
            input_dim=X_tr.shape[1],
            hidden_units=hidden_units,
            activation=activation,
            dropout=dropout,
            batch_norm=batch_norm,
            optimizer_name=optimizer_name,
            lr=lr,
            weight_decay=weight_decay,
        )

        cb = tf.keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=patience, restore_best_weights=True
        )

        model.fit(
            X_tr, y_tr,
            validation_data=(X_va, y_va),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[cb],
            verbose=verbose
        )

        proba = model.predict(X_va, verbose=0).ravel()
        pred = (proba >= 0.5).astype(int)
        scores.append(f1_score(y_va, pred))

        tf.keras.backend.clear_session()

    scores = np.array(scores, dtype=float)
    return float(scores.mean()), float(scores.std())


In [ ]:
import itertools
import pandas as pd

# --- pick the right feature matrix automatically ---
# (this avoids the X_encoded vs X_encoded typo problem)
for name in ["X_encoded", "X_encoded_fg", "X_inter", "X_alone"]:
    if name in globals():
        X_nn = globals()[name]
        X_nn_name = name
        break
else:
    raise NameError("Couldn't find any of: X_encoded, X_encoded_fg, X_inter, X_alone. "
                    "Make sure you created your encoded feature matrix first.")

print("Using feature matrix:", X_nn_name, "shape:", X_nn.shape)

# --- COARSE grid (big-picture spacing, not too huge) ---
grid = {
    "optimizer_name": ["adam", "adamw", "sgd"],
    "lr": [3e-3, 1e-3, 3e-4],
    "hidden_units": [(64,), (128, 64), (256, 128, 64)],
    "dropout": [0.0, 0.3, 0.5],
    "activation": ["relu", "gelu", "leakyrelu"],
    "batch_norm": [True],
}

fixed = dict(
    epochs=80,
    batch_size=64,
    patience=10,
    n_splits=5,
    random_state=42,
    verbose=0,
)

results = []
keys = list(grid.keys())

for values in itertools.product(*[grid[k] for k in keys]):
    cfg = dict(zip(keys, values))
    mean_f1, std_f1 = eval_nn_cv_f1(X_nn, y, **cfg, **fixed)
    row = {**cfg, "mean_f1": mean_f1, "std_f1": std_f1}
    results.append(row)
    print(f"done: {cfg} -> {mean_f1:.4f} ± {std_f1:.4f}")

df_res = pd.DataFrame(results).sort_values(["mean_f1", "std_f1"], ascending=[False, True]).reset_index(drop=True)
df_res.head(15)
